## Import

In [73]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

import glob, os

## Utilities

In [53]:
def rGetFiles(dirName, fName):
    fList = []
    count = 0
    for root, dirs, files in os.walk(dirName):
        for file in files:
            if file == fName:
                #print(os.path.join(root, file))
                fList.append(os.path.join(root, file))
                count = count + 1
        #end-for
    #end-for
    print("Total matched files : %d" % (count))
    return fList
#end-rGetFiles

In [113]:
def sepFileList(fList):
    eeg_count = 0
    nirs_count = 0
    eeg_list = []
    nirs_list = []
    for f in fList:
        if f.rfind("-EEG") >= 0:
            eeg_list.append(f)
            eeg_count = eeg_count + 1
        elif f.rfind("-NIRS") >= 0:
            nirs_list.append(f)
            nirs_count = nirs_count + 1
    #end-for
    print("EEG Files : %d, NIRS Files : %d" % (eeg_count, nirs_count))
    return eeg_list, nirs_list

In [114]:
def getParentDirName(path):
    path_str = os.path.abspath(path).split('/')
    par_name = path_str[len(path_str)-2]
    return par_name
#end-getParentDirName

In [116]:
def getParentDir(fPath):
    return os.path.abspath(os.path.join(fPath, os.pardir))
#end-getParentDir

In [117]:
def moveFile(fName, destDir):
    os.rename(fName, destDir + "/" + fName)
#end-moveFile

## Get data files

In [91]:
input_files = [ 'cnt_dsr' ,'mnt_dsr', 'mrk_dsr']
extn = ".mat"

getFiles(eegFiles, input_files[0]+extn)

print("cnt_dsr")
files = rGetFiles(os.curdir, input_files[0] + extn)
eeg_cnt_dsr_flist = []
nirs_cnt_dsr_flist = []
eeg_cnt_dsr_flist, nirs_cnt_dsr_flist = sepFileList(files)
 
print("mnt_dsr")
files = rGetFiles(os.curdir, input_files[1] + extn)
eeg_mnt_dsr_flist = []
nirs_mnt_dsr_flist = []
eeg_mnt_dsr_flist, nirs_mnt_dsr_flist = sepFileList(files)

print("mrk_dsr")
files = rGetFiles(os.curdir, input_files[2] + extn)
eeg_mrk_dsr_flist = []
nirs_mrk_dsr_flist = []
eeg_mrk_dsr_flist, nirs_mrk_dsr_flist = sepFileList(files)

cnt_dsr
Total matched files : 52
EEG Files : 26, NIRS Files : 26
mnt_dsr
Total matched files : 52
EEG Files : 26, NIRS Files : 26
mrk_dsr
Total matched files : 52
EEG Files : 26, NIRS Files : 26


## Extract and Export cnt_dsr Matlab data to CSV

In [3]:
def extract_cnt_dsr(fName):
    #print("file : ", fName)
    data = loadmat(fName, squeeze_me=True)
    result = []
    for oxyordeoxy in data['cnt_dsr'].tolist():
        result.append([])
        for data in oxyordeoxy.tolist():
            if (type(data) is np.ndarray):
                print('data ={}'.format(data.shape))
                result[i].append(data)
        #end-for
    #end-for
    return result
#end-extract_cnt_dsr

In [120]:
def export_to_csv(result, correspondingFile):
    #print("file : ", correspondingFile)
    f = correspondingFile.split('/')
    oxydata =  pd.DataFrame(data=result[0][1].tolist(), columns=result[0][0])
    #print(oxydata.head)
    oxy_file = f[len(f)-1].strip(extn) + '_oxydata.csv'
    oxydata.to_csv(oxy_file)
    moveFile(oxy_file, getParentDir(correspondingFile))
    
    deoxy_file = f[len(f)-1].strip(extn) + '_deoxydata.csv'
    deoxydata =  pd.DataFrame(data=result[1][1].tolist(), columns=result[1][0])
    deoxydata.to_csv(deoxy_file)
    moveFile(deoxy_file, getParentDir(correspondingFile))
#end-export_to_csv

## Extract and Export mrk_dsr Matlab data to CSV

In [136]:
def extract_mrk_dsr(fName):
    data = scipy.io.loadmat(fName, struct_as_record=True)
    result = []

    for arr in data['mrk_dsr'].tolist():       
        for onedata in arr:
            print('loop')
            #result.append(onedata)
            for oned in onedata:
                for one in oned:
                    for t in one:
                        print('t={0}->{1}'.format(type(t), t))
                        result.append(t)
                    #end-for
                #end-for
            #end-for
        #end-for
    #end-for
    return result
#end-extract_mnt_dsr

In [152]:
def export_mrk_dsr_to_csv(result, correspondingFile):
    #print(correspondingFile)
    f = correspondingFile.split('/')
    #print(f)
    fName = f[len(f)-1].split('.')[0] + '.csv'
    #print('exporting with filename : %s' % (fName))
    data =  pd.DataFrame(data=result)        
    #print(data.shape)
    data.to_csv(fName, index=False)
    moveFile(fName, getParentDir(correspondingFile))

## Extract and Export mnt_dsr Matlab data to CSV

In [160]:
def extract_mnt_dsr(fName):
    data = scipy.io.loadmat(fName, struct_as_record=True)
    result =[]

    for oxyordeoxy in data['mnt_dsr'].tolist():    
        for onedata in oxyordeoxy:
            result.append(onedata)
        #end-for
    #end-for
#end-extract_mnt_dsr

In [161]:
def export_mnt_dsr_to_csv(result, correspondingFile):
    #print(correspondingFile)
    f = correspondingFile.split('/')
    #print(f)
    fName = f[len(f)-1].split('.')[0] + '.csv'
    #print('exporting with filename : %s' % (fName))
    data =  pd.DataFrame(data=result)        
    #print(data.shape)
    data.to_csv(fName, index=False)
    moveFile(fName, getParentDir(correspondingFile))

## MATLAB to CSV

In [2]:
def mat_to_csv(dataStr):
    print('mat_to_csv for %s' % (dataStr))
    if dataStr == input_files[0]:
        for file_index in nirs_cnt_dsr_flist:
            res = extract_cnt_dsr(file_index)
            export_to_csv(res, file_index)
    elif dataStr == input_files[1]:
        for file_index in nirs_mnt_dsr_flist:
            res = extract_mnt_dsr(file_index)
            export_mnt_dsr_to_csv(res, file_index)
    elif dataStr == input_files[2]:
        for file_index in nirs_mrk_dsr_flist:
            res = extract_mrk_dsr(file_index)
            export_mrk_dsr_to_csv(res, file_index)
#end-mat_to_csv

In [ ]:
mat_to_csv(input_files[0])
mat_to_csv(input_files[1])
mat_to_csv(input_files[2])